In [1]:
import pandas as pd
import numpy as np

In [2]:
import json

with open ("transaction-data-adhoc-analysis.json", "r") as f:
    data = json.load(f)
    
Lola_Tamis = "transaction-data-adhoc-analysis.json"
df = pd.read_json(Lola_Tamis)
df.head()

,address,birthdate,mail,name,sex,username,transaction_items,transaction_value,transaction_date
0,USNS Wagner\nFPO AE 00917,1926/03/11,watsonbrenda@gmail.com,Jennifer Campbell,F,veronicawells,"Exotic Extras,Beef Chicharon,(x4)",5196,2022/01/12
1,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",12056,2022/01/12
2,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"HealthyKid 3+,Gummy Vitamins,(x3);HealthyKid 3...",5500,2022/01/08
3,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"HealthyKid 3+,Yummy Vegetables,(x1)",500,2022/01/10
4,"33911 Hansen Loaf\nWest Sandra, ND 54188",1949/09/27,michael78@yahoo.com,Kristen Cooper,F,qhernandez,"Candy City,Orange Beans,(x3);Candy City,Gummy ...",2697,2022/01/24


COUNT ITEMS PER MONTH

In [3]:
food_df = df[['transaction_items','transaction_date']].copy()
food_df     

,transaction_items,transaction_date
0,"Exotic Extras,Beef Chicharon,(x4)",2022/01/12
1,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",2022/01/12
2,"HealthyKid 3+,Gummy Vitamins,(x3);HealthyKid 3...",2022/01/08
3,"HealthyKid 3+,Yummy Vegetables,(x1)",2022/01/10
4,"Candy City,Orange Beans,(x3);Candy City,Gummy ...",2022/01/24
...,...,...
83031,"Exotic Extras,Kimchi and Seaweed,(x2);HealthyK...",2022/06/05
83032,"HealthyKid 3+,Yummy Vegetables,(x1);Exotic Ext...",2022/06/12
83033,"Candy City,Orange Beans,(x4);Candy City,Gummy ...",2022/06/15
83034,"HealthyKid 3+,Nutrional Milk,(x2);Candy City,G...",2022/06/09


In [4]:
#adding each month 

food_df["January"]=food_df["transaction_date"].str.contains ("01/")
food_df["February"]=food_df["transaction_date"].str.contains ("02/")
food_df["March"]=food_df["transaction_date"].str.contains ("03/")
food_df["April"]=food_df["transaction_date"].str.contains ("04/")
food_df["May"]=food_df["transaction_date"].str.contains ("05/")
food_df["June"]=food_df["transaction_date"].str.contains ("06/")

food_df[['transaction_items', 'transaction_date', 'January', 'February', 'March','April', 'May', 'June']]

,transaction_items,transaction_date,January,February,March,April,May,June
0,"Exotic Extras,Beef Chicharon,(x4)",2022/01/12,True,False,False,False,False,False
1,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",2022/01/12,True,False,False,False,False,False
2,"HealthyKid 3+,Gummy Vitamins,(x3);HealthyKid 3...",2022/01/08,True,False,False,False,False,False
3,"HealthyKid 3+,Yummy Vegetables,(x1)",2022/01/10,True,False,False,False,False,False
4,"Candy City,Orange Beans,(x3);Candy City,Gummy ...",2022/01/24,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...
83031,"Exotic Extras,Kimchi and Seaweed,(x2);HealthyK...",2022/06/05,False,False,False,False,False,True
83032,"HealthyKid 3+,Yummy Vegetables,(x1);Exotic Ext...",2022/06/12,False,False,False,False,False,True
83033,"Candy City,Orange Beans,(x4);Candy City,Gummy ...",2022/06/15,False,False,False,False,False,True
83034,"HealthyKid 3+,Nutrional Milk,(x2);Candy City,G...",2022/06/09,False,False,False,False,False,True


In [5]:
#creating functions for monthly dataframes, counting food
def monthly_dataframe (month):
    
    is_food = food_df[month]== True
    food_df[is_food] 
    month_food = food_df[is_food].copy()
    month_food.drop(columns = ['January', 'February', 'March','April', 'May', 'June'], inplace = True)
    
    #split transaction items by ";"
    rm_levels = month_food["transaction_items"].str.split(";", expand=True).stack().reset_index()
    rm_levels.drop(columns = ['level_0', 'level_1'], inplace = True)
    
    #split transaction items by ","
    food_heading = rm_levels[0].str.split(",", expand=True)
    
    #adding each food type 
    food_heading["Beef_Chicharon"]=food_heading[1].str.contains ("Beef Chicharon")
    food_heading["Kimchi_and_Seaweed"]=food_heading[1].str.contains ("Kimchi and Seaweed")
    food_heading["Yummy_Vegetables"]=food_heading[1].str.contains ("Yummy Vegetables")
    food_heading["Gummy_Vitamins"]=food_heading[1].str.contains ("Gummy Vitamins")
    food_heading["Nutrional_Milk"]=food_heading[1].str.contains ("Nutrional Milk")
    food_heading["Orange_Beans"]=food_heading[1].str.contains ("Orange Beans")
    food_heading["Gummy_Worms"]=food_heading[1].str.contains ("Gummy Worms")

    food_heading[[                   0,                    1,                    2,
               'Beef_Chicharon', 'Kimchi_and_Seaweed',   'Yummy_Vegetables',
               'Gummy_Vitamins',   'Nutrional_Milk',       'Orange_Beans',
                  'Gummy_Worms']]
    return food_heading

def count_food(food, monthly_df):
    total_food = 0
    for i in range(len(monthly_df)):
        if monthly_df[food][i]: 
            total_food += int(monthly_df[2][i][2])
    return total_food

In [6]:
#january - count food
january_count_df = monthly_dataframe ("January")

bc1 = count_food("Beef_Chicharon", january_count_df)
ks1 = count_food("Kimchi_and_Seaweed", january_count_df)
yv1 = count_food("Yummy_Vegetables", january_count_df)
gv1 = count_food("Gummy_Vitamins", january_count_df)
nm1 = count_food("Nutrional_Milk", january_count_df)
ob1 = count_food("Orange_Beans", january_count_df)
gw1 = count_food("Gummy_Worms", january_count_df)

print (bc1,ks1,yv1,gv1,nm1,ob1,gw1)

9665 9676 9959 9681 9727 9774 9559


In [7]:
#february - count food
february_count_df = monthly_dataframe ("February")

bc2 = count_food("Beef_Chicharon", february_count_df)
ks2 = count_food("Kimchi_and_Seaweed", february_count_df)
yv2 = count_food("Yummy_Vegetables", february_count_df)
gv2 = count_food("Gummy_Vitamins", february_count_df)
nm2 = count_food("Nutrional_Milk", february_count_df)
ob2 = count_food("Orange_Beans", february_count_df)
gw2 = count_food("Gummy_Worms", february_count_df)

print (bc2,ks2,yv2,gv2,nm2,ob2,gw2)

10001 9949 10256 9980 9691 10037 9996


In [8]:
#march - count food
march_count_df = monthly_dataframe ("March")

bc3 = count_food("Beef_Chicharon", march_count_df)
ks3 = count_food("Kimchi_and_Seaweed", march_count_df)
yv3 = count_food("Yummy_Vegetables", march_count_df)
gv3 = count_food("Gummy_Vitamins", march_count_df)
nm3 = count_food("Nutrional_Milk", march_count_df)
ob3 = count_food("Orange_Beans", march_count_df)
gw3 = count_food("Gummy_Worms", march_count_df)

print (bc3,ks3,yv3,gv3,nm3,ob3,gw3)

9816 9967 9896 10145 9876 9611 9986


In [9]:
#april - count food
april_count_df = monthly_dataframe ("April")

bc4 = count_food("Beef_Chicharon", april_count_df)
ks4 = count_food("Kimchi_and_Seaweed", april_count_df)
yv4 = count_food("Yummy_Vegetables", april_count_df)
gv4 = count_food("Gummy_Vitamins", april_count_df)
nm4 = count_food("Nutrional_Milk", april_count_df)
ob4 = count_food("Orange_Beans", april_count_df)
gw4 = count_food("Gummy_Worms", april_count_df)

print (bc4,ks4,yv4,gv4,nm4,ob4,gw4)

9890 9921 9861 9842 9786 9914 10043


In [10]:
#may - count food
may_count_df = monthly_dataframe ("May")

bc5 = count_food("Beef_Chicharon", may_count_df)
ks5 = count_food("Kimchi_and_Seaweed", may_count_df)
yv5 = count_food("Yummy_Vegetables", may_count_df)
gv5 = count_food("Gummy_Vitamins", may_count_df)
nm5 = count_food("Nutrional_Milk", may_count_df)
ob5 = count_food("Orange_Beans", may_count_df)
gw5 = count_food("Gummy_Worms", may_count_df)

print (bc5,ks5,yv5,gv5,nm5,ob5,gw5)

10028 9773 9735 9948 9881 9964 9801


In [11]:
#june - count food
june_count_df = monthly_dataframe ("June")

bc6 = count_food("Beef_Chicharon", june_count_df)
ks6 = count_food("Kimchi_and_Seaweed", june_count_df)
yv6 = count_food("Yummy_Vegetables", june_count_df)
gv6 = count_food("Gummy_Vitamins", june_count_df)
nm6 = count_food("Nutrional_Milk", june_count_df)
ob6 = count_food("Orange_Beans", june_count_df)
gw6 = count_food("Gummy_Worms", june_count_df)

print (bc6,ks6,yv6,gv6,nm6,ob6,gw6)

9902 10104 9722 9980 9767 10106 9934


In [12]:
# function for monthly pivot table
def monthly_pivot_count (month, bc, ks, yv, gv, nm, ob, gw):
    monthly_dic = {"Beef_Chicharon": [bc], 
                "Kimchi_and_Seaweed": [ks], 
                "Yummy_Vegetables": [yv], 
                "Gummy_Vitamins": [gv], 
                "Nutrional_Milk": [nm], 
                "Orange_Beans": [ob], 
                "Gummy_Worms" : [gw]
               }

    #turn into its own pivot table
    monthly_df = pd.DataFrame(monthly_dic, index=[month])
    
    return monthly_df

In [13]:
#monthly pivot tables
January_df = monthly_pivot_count ("January", bc1, ks1, yv1, gv1, nm1, ob1, gw1)
February_df = monthly_pivot_count ("February", bc2, ks2, yv2, gv2, nm2, ob2, gw2)
March_df = monthly_pivot_count ("March", bc3, ks3, yv3, gv3, nm3, ob3, gw3)
April_df = monthly_pivot_count ("April", bc4, ks4, yv4, gv4, nm4, ob4, gw4)
May_df = monthly_pivot_count ("May", bc5, ks5, yv5, gv5, nm5, ob5, gw5)
June_df = monthly_pivot_count ("June", bc6, ks6, yv6, gv6, nm6, ob6, gw6)

#concat all table
pd.concat([January_df,February_df,March_df,April_df,May_df,June_df])

,Beef_Chicharon,Kimchi_and_Seaweed,Yummy_Vegetables,Gummy_Vitamins,Nutrional_Milk,Orange_Beans,Gummy_Worms
January,9665,9676,9959,9681,9727,9774,9559
February,10001,9949,10256,9980,9691,10037,9996
March,9816,9967,9896,10145,9876,9611,9986
April,9890,9921,9861,9842,9786,9914,10043
May,10028,9773,9735,9948,9881,9964,9801
June,9902,10104,9722,9980,9767,10106,9934


TOTAL SALE VALUE PER ITEM PER MONTH

In [14]:
#get transaction value per item-- manually from data 
    #sample data from file:
        #"transaction_items": "Exotic Extras,Beef Chicharon,(x4)"; "transaction_value": 5196
        #"transaction_items": "Exotic Extras,Kimchi and Seaweed,(x4)"; "transaction_value": 3196
        #"transaction_items": "HealthyKid 3+,Yummy Vegetables,(x3)"; "transaction_value": 1500
        #"transaction_items": "HealthyKid 3+,Gummy Vitamins,(x2)"; "transaction_value": 3000
        #"transaction_items": "HealthyKid 3+,Nutrional Milk,(x3)";"transaction_value": 5970
        #"transaction_items": "Candy City,Orange Beans,(x3)"; "transaction_value": 597
        #transaction_items": "Candy City,Gummy Worms,(x1)"; "transaction_value": 150

vbc = 5196/4
vks = 3196/4
vyv = 1500/3
vgv = 3000/2
vnm = 5970/3
vob = 597/3
vgw = 150

#sale value per item
def value_price_dic(bc, ks, yv, gv, nm, ob, gw):
    total_bc = bc*vbc
    total_ks = ks*vks
    total_yv = yv*vyv
    total_gv = gv*vgv
    total_nm = nm*vnm
    total_ob = ob*vob
    total_gw = gw*vgw
    
    return total_bc,total_ks,total_yv, total_gv, total_nm, total_ob, total_gw

In [15]:
#monthly - sale value per item
jan_value = value_price_dic(bc1, ks1, yv1, gv1, nm1, ob1, gw1)
feb_value = value_price_dic(bc2, ks2, yv2, gv2, nm2, ob2, gw2)
mar_value = value_price_dic(bc3, ks3, yv3, gv3, nm3, ob3, gw3)
apr_value = value_price_dic(bc4, ks4, yv4, gv4, nm4, ob4, gw4)
may_value = value_price_dic(bc5, ks5, yv5, gv5, nm5, ob5, gw5)
jun_value = value_price_dic(bc6, ks6, yv6, gv6, nm6, ob6, gw6)

In [16]:
#function for monthly pivot table
def monthly_pivot_value (monthly_sale_value_per_item, month):
    items = ("Beef_Chicharon", "Kimchi_and_Seaweed", "Yummy_Vegetables", "Gummy_Vitamins", "Nutrional_Milk", "Orange_Beans", "Gummy_Worms")
    value_dic = {items[i]: monthly_sale_value_per_item[i] for i in range(len(items))}
    value_df = pd.DataFrame(value_dic, index=[month])
    return value_df.astype(int)

In [17]:
#monthly pivot tables
January_value_df = monthly_pivot_value (jan_value, "January")
February_value_df = monthly_pivot_value (feb_value, "February")
March_value_df = monthly_pivot_value (mar_value, "March")
April_value_df = monthly_pivot_value (apr_value, "April")
May_value_df = monthly_pivot_value (may_value, "May")
June_value_df = monthly_pivot_value (jun_value, "June")

#concat all table
pd.concat([January_value_df,February_value_df,March_value_df,April_value_df,May_value_df,June_value_df])

,Beef_Chicharon,Kimchi_and_Seaweed,Yummy_Vegetables,Gummy_Vitamins,Nutrional_Milk,Orange_Beans,Gummy_Worms
January,12554835,7731124,4979500,14521500,19356730,1945026,1433850
February,12991299,7949251,5128000,14970000,19285090,1997363,1499400
March,12750984,7963633,4948000,15217500,19653240,1912589,1497900
April,12847110,7926879,4930500,14763000,19474140,1972886,1506450
May,13026372,7808627,4867500,14922000,19663190,1982836,1470150
June,12862698,8073096,4861000,14970000,19436330,2011094,1490100


LOLA TAMIS'S TABLE METRICS

In [18]:
metrics_df = df[['username','transaction_date']].copy()
metrics_df

,username,transaction_date
0,veronicawells,2022/01/12
1,cgilbert,2022/01/12
2,cgilbert,2022/01/08
3,cgilbert,2022/01/10
4,qhernandez,2022/01/24
...,...,...
83031,kmcconnell,2022/06/05
83032,llowe,2022/06/12
83033,llowe,2022/06/15
83034,llowe,2022/06/09


In [19]:
#adding each month 

metrics_df["January"]=metrics_df["transaction_date"].str.contains ("01/")
metrics_df["February"]=metrics_df["transaction_date"].str.contains ("02/")
metrics_df["March"]=metrics_df["transaction_date"].str.contains ("03/")
metrics_df["April"]=metrics_df["transaction_date"].str.contains ("04/")
metrics_df["May"]=metrics_df["transaction_date"].str.contains ("05/")
metrics_df["June"]=metrics_df["transaction_date"].str.contains ("06/")

metrics_df[['username', 'transaction_date', 'January', 'February', 'March','April', 'May', 'June']]

,username,transaction_date,January,February,March,April,May,June
0,veronicawells,2022/01/12,True,False,False,False,False,False
1,cgilbert,2022/01/12,True,False,False,False,False,False
2,cgilbert,2022/01/08,True,False,False,False,False,False
3,cgilbert,2022/01/10,True,False,False,False,False,False
4,qhernandez,2022/01/24,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...
83031,kmcconnell,2022/06/05,False,False,False,False,False,True
83032,llowe,2022/06/12,False,False,False,False,False,True
83033,llowe,2022/06/15,False,False,False,False,False,True
83034,llowe,2022/06/09,False,False,False,False,False,True


In [20]:
#separating dataframes per month
def monthly_dataframe_metrics (month):
    
    is_metrics = metrics_df[month]== True
    metrics_df[is_metrics] 
    month_metrics = metrics_df[is_metrics].copy()
    month_metrics.drop(columns = ['January', 'February', 'March','April', 'May', 'June'], inplace = True)
    
    return month_metrics

In [21]:
#monthly dataframe
January_metrics = monthly_dataframe_metrics ("January")
February_metrics = monthly_dataframe_metrics ("February")
March_metrics = monthly_dataframe_metrics ("March")
April_metrics = monthly_dataframe_metrics ("April")
May_metrics = monthly_dataframe_metrics ("May")
June_metrics = monthly_dataframe_metrics ("June")

In [22]:
#function for repeaters 
def repeaters_metric (previous_month, current_month):
    rm_duplicates1 = previous_month["username"].unique()
    series1 = pd.Series(rm_duplicates1) 
    rm_duplicates2 = current_month["username"].unique()
    series2 = pd.Series(rm_duplicates2) 
    return pd.Series(series2.isin(series1).values.astype(int), series2.values)

In [23]:
#repeaters per month; no january as it = 0
feb_repeaters = repeaters_metric (January_metrics, February_metrics)
feb_repeaters_metric = feb_repeaters.value_counts()[1]

mar_repeaters = repeaters_metric (February_metrics, March_metrics)
mar_repeaters_metric = mar_repeaters.value_counts()[1]

apr_repeaters = repeaters_metric (March_metrics, April_metrics)
apr_repeaters_metric = apr_repeaters.value_counts()[1]

may_repeaters = repeaters_metric (April_metrics, May_metrics)
may_repeaters_metric = may_repeaters.value_counts()[1]

jun_repeaters = repeaters_metric (May_metrics, June_metrics)
jun_repeaters_metric = jun_repeaters.value_counts()[1]

In [24]:
#turn into dictionary for pivot table
repeater_dic = {"January": [0], 
            "February": [feb_repeaters_metric], 
            "March": [mar_repeaters_metric], 
            "April": [apr_repeaters_metric], 
            "May": [may_repeaters_metric], 
            "June": [jun_repeaters_metric], 
           }

#turn into pivot table
repeaters_df = pd.DataFrame(repeater_dic, index=["Repeaters"])
repeaters_df

,January,February,March,April,May,June
Repeaters,0,5167,5239,5191,5126,5208


In [25]:
#inactive; january holds none (0)
feb_inactive_metric = feb_repeaters.value_counts()[0] 

In [26]:
march_inactive = pd.concat([January_metrics, February_metrics]) 
unique_march = march_inactive["username"].drop_duplicates(keep = "last")

#compares metrics from jan+feb with march
march_concat = pd.concat([unique_march,March_metrics["username"]]) 

#ensures all first time buyers are removed in current month
march_concat = pd.concat([unique_march,March_metrics["username"]])

march_final = march_concat.drop_duplicates(keep=False)
mar_inactive_metric = march_final.count()

In [27]:
april_inactive = pd.concat([January_metrics, February_metrics, March_metrics]) 
unique_april = april_inactive["username"].drop_duplicates(keep = "last")

#compares metrics from jan+feb+march with april
april_concat = pd.concat([unique_april,April_metrics["username"]]) 

#ensures all first time buyers are removed in current month
april_concat = pd.concat([unique_april,April_metrics["username"]])

april_final = april_concat.drop_duplicates(keep=False)
apr_inactive_metric = april_final.count()

In [28]:
may_inactive = pd.concat([January_metrics, February_metrics, March_metrics, April_metrics]) 
unique_may = may_inactive["username"].drop_duplicates(keep = "last")

#compares metrics from jan+feb+march+april with may
may_concat = pd.concat([unique_may,May_metrics["username"]]) 

#ensures all first time buyers are removed in current month
may_concat = pd.concat([unique_may,May_metrics["username"]]) 

may_final = may_concat.drop_duplicates(keep=False)
may_inactive_metric = may_final.count()

In [29]:
june_inactive = pd.concat([January_metrics, February_metrics, March_metrics, April_metrics, May_metrics]) 
unique_june = june_inactive["username"].drop_duplicates(keep = "last")

#compares metrics from jan+feb with march
june_concat = pd.concat([unique_june,June_metrics["username"]]) 

#ensures all first time buyers are removed in current month
june_concat = pd.concat([unique_june,June_metrics["username"]]) 

june_final = june_concat.drop_duplicates(keep=False)
jun_inactive_metric = june_final.count()

In [30]:
#turn into dictionary for pivot table
inactive_dic = {"January": [0], 
            "February": [feb_inactive_metric], 
            "March": [mar_inactive_metric], 
            "April": [apr_inactive_metric], 
            "May": [may_inactive_metric], 
            "June": [jun_inactive_metric], 
           }

#turn into pivot table
inactive_df = pd.DataFrame(inactive_dic, index=["Inactive"])
inactive_df

,January,February,March,April,May,June
Inactive,0,1463,1824,1939,1920,1840


In [31]:
#engaged january
jan_engaged = January_metrics["username"].drop_duplicates(keep= "last")
jan_engaged_metric = jan_engaged.count()

In [32]:
#engaged february
feb_engaged_metric = feb_repeaters_metric

In [33]:
#engaged march

#get usernames from jan&feb
january_unique_username = January_metrics["username"].unique()
january_unique_username_df = pd.DataFrame(january_unique_username) 
january_unique_username_df.columns = ["username"]

february_unique_username = February_metrics["username"].unique()
february_unique_username_df = pd.DataFrame(february_unique_username) 
february_unique_username_df.columns = ["username"]

#find feb users inside jan
jan_feb_total = february_unique_username_df.assign(presence=february_unique_username_df.username.isin(january_unique_username_df.username).astype(int))
total_jan_feb = jan_feb_total.loc[jan_feb_total["presence"]==1]

#get usernames from march
march_unique_username = March_metrics["username"].unique()
march_unique_username_df = pd.DataFrame(march_unique_username) 
march_unique_username_df.columns = ["username"]

#find march users inside jan+feb
march_inside_total_jan_feb = march_unique_username_df.assign(presence=march_unique_username_df.username.isin(total_jan_feb.username).astype(int))
mar_engaged = march_inside_total_jan_feb.loc[march_inside_total_jan_feb["presence"]==1]
mar_engaged_metric = mar_engaged.count()[1]

In [34]:
#engaged april

#list of usernames from jan-feb-mar
mar_engaged_metric

#get usernames from april
april_unique_username = April_metrics["username"].unique()
april_unique_username_df = pd.DataFrame(april_unique_username) 
april_unique_username_df.columns = ["username"]

#find april users inside jan+feb+mar
april_inside_total_jan_feb_mar = april_unique_username_df.assign(presence=april_unique_username_df.username.isin(mar_engaged.username).astype(int))
apr_engaged = april_inside_total_jan_feb_mar.loc[april_inside_total_jan_feb_mar["presence"]==1]
apr_engaged_metric = apr_engaged.count()[1]

In [35]:
#engaged may

#list of usernames from jan-feb-mar-apr
apr_engaged_metric

#get usernames from april
may_unique_username = May_metrics["username"].unique()
may_unique_username_df = pd.DataFrame(may_unique_username) 
may_unique_username_df.columns = ["username"]
may_unique_username_df

#find april users inside jan+feb+mar
may_inside_total_jan_feb_mar_apr = may_unique_username_df.assign(presence=may_unique_username_df.username.isin(apr_engaged.username).astype(int))
may_engaged = may_inside_total_jan_feb_mar_apr.loc[may_inside_total_jan_feb_mar_apr["presence"]==1]
may_engaged_metric = may_engaged.count()[1]

In [36]:
#engaged june

#list of usernames from jan-feb-mar-apr
may_engaged_metric

#get usernames from april
june_unique_username = June_metrics["username"].unique()
june_unique_username_df = pd.DataFrame(june_unique_username) 
june_unique_username_df.columns = ["username"]
june_unique_username_df

#find april users inside jan+feb+mar
june_inside_total_jan_feb_mar_apr_may = june_unique_username_df.assign(presence=june_unique_username_df.username.isin(may_engaged.username).astype(int))
jun_engaged = june_inside_total_jan_feb_mar_apr_may.loc[june_inside_total_jan_feb_mar_apr_may["presence"]==1]
jun_engaged_metric = jun_engaged.count()[1]

In [37]:
#turn into dictionary for pivot table
engaged_dic = {"January": [jan_engaged_metric], 
            "February": [feb_engaged_metric], 
            "March": [mar_engaged_metric], 
            "April": [apr_engaged_metric], 
            "May": [may_engaged_metric], 
            "June": [jun_engaged_metric], 
           }

#turn into pivot table
engaged_df = pd.DataFrame(engaged_dic, index=["Engaged"])
engaged_df

,January,February,March,April,May,June
Engaged,6583,5167,4141,3304,2685,2210


In [38]:
#combine into pivot
pd.concat([repeaters_df,inactive_df,engaged_df])

,January,February,March,April,May,June
Repeaters,0,5167,5239,5191,5126,5208
Inactive,0,1463,1824,1939,1920,1840
Engaged,6583,5167,4141,3304,2685,2210


Additional Metrics

In [39]:
#number of unique customers per month
January_customers = January_metrics.nunique()[0]
February_customers = February_metrics.nunique()[0]
March_customers = March_metrics.nunique()[0]
April_customers = April_metrics.nunique()[0]
May_customers = May_metrics.nunique()[0]
June_customers = June_metrics.nunique()[0]

total_customers_per_month = {"January": [January_customers], 
            "February": [February_customers], 
            "March": [March_customers], 
            "April": [April_customers], 
            "May": [May_customers], 
            "June": [June_customers], 
           }

#turn into pivot table
num_customers_monthly = pd.DataFrame(total_customers_per_month, index=["Overall Customer Transactions"])

In [40]:
#Count of New Customers

#january
jan_new_metric = january_unique_username_df.count()[0]

#february
feb_not_in_jan = february_unique_username_df.assign(presence=february_unique_username_df.username.isin(january_unique_username_df.username).astype(int))
total_feb_not_in_jan = feb_not_in_jan.loc[feb_not_in_jan["presence"]==0]
feb_new_metric = total_feb_not_in_jan.count()[1]

#march
march_not_in_total_jan_feb = total_feb_not_in_jan.assign(presence=total_feb_not_in_jan.username.isin(march_unique_username_df.username).astype(int))
total_mar_not_in_jan_feb = march_not_in_total_jan_feb.loc[march_not_in_total_jan_feb["presence"]==0]
mar_new_metric = total_mar_not_in_jan_feb.count()[1]

#april
april_not_in_total_jan_feb_mar = total_mar_not_in_jan_feb.assign(presence=total_mar_not_in_jan_feb.username.isin(april_unique_username_df.username).astype(int))
total_apr_not_in_jan_feb_mar = april_not_in_total_jan_feb_mar.loc[april_not_in_total_jan_feb_mar["presence"]==0]
apr_new_metric = total_apr_not_in_jan_feb_mar.count()[1]

#may
may_not_in_total_jan_feb_mar_apr = total_apr_not_in_jan_feb_mar.assign(presence=total_apr_not_in_jan_feb_mar.username.isin(may_unique_username_df.username).astype(int))
total_may_not_in_jan_feb_mar_apr = may_not_in_total_jan_feb_mar_apr.loc[may_not_in_total_jan_feb_mar_apr["presence"]==0]
may_new_metric = total_may_not_in_jan_feb_mar_apr.count()[1]

#june
jun_not_in_total_jan_feb_mar_apr_may = total_may_not_in_jan_feb_mar_apr.assign(presence=total_may_not_in_jan_feb_mar_apr.username.isin(june_unique_username_df.username).astype(int))
total_june_not_in_jan_feb_mar_apr_may = jun_not_in_total_jan_feb_mar_apr_may.loc[jun_not_in_total_jan_feb_mar_apr_may["presence"]==0]
jun_new_metric = total_june_not_in_jan_feb_mar_apr_may.count()[1]

In [41]:
#turn into pivot table
total_newcustomers_per_month = {"January": [jan_new_metric], 
            "February": [feb_new_metric], 
            "March": [mar_new_metric], 
            "April": [apr_new_metric], 
            "May": [may_new_metric], 
            "June": [jun_new_metric], 
           }

#turn into pivot table
num_new_customers_monthly = pd.DataFrame(total_newcustomers_per_month, index=["New Customers"])

Additional Metrics: Customer Transactions Table

In [42]:
pd.concat([num_customers_monthly,num_new_customers_monthly])

,January,February,March,April,May,June
Overall Customer Transactions,6583,6630,6658,6568,6587,6665
New Customers,6583,1463,365,113,26,2
